In [1]:
import datetime 
#from gensim.models import Word2Vec 
from itertools import combinations 
import random 
import pandas as pd 
from sklearn.calibration import label_binarize 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, f1_score, roc_auc_score 
from sklearn.decomposition import LatentDirichletAllocation 
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.manifold import TSNE 
import numpy as np 
import re 
import string 
from transformers import BertTokenizer, BertModel 
#import shap 
import matplotlib.pyplot as plt 
from imblearn.over_sampling import SMOTE 
import seaborn as sns 
from ast import literal_eval
#from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_predict
from langdetect import detect
import pickle

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
import pandas as pd
import numpy as np
import spacy
import re
import string
from gensim.models import Word2Vec
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from imblearn.over_sampling import SMOTE

In [3]:
comments_df = pd.read_csv('comments_f2v_fts.csv')
videos_df = pd.read_csv('vid_fin.csv')

In [4]:
comments_df = comments_df[:1000]

In [5]:
# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø–æ–¥—Å—á–µ—Ç–∞ –∏–Ω–æ—Å—Ç—Ä–∞–Ω–Ω—ã—Ö —Å–ª–æ–≤
def count_foreign_words(text):
    if isinstance(text, str):
        foreign_word_count = 0
        words = text.split()
        for word in words:
            try:
                lang = detect(word)
                if lang != 'ru':
                    foreign_word_count += 1
            except LangDetectException:
                continue  # –ü—Ä–æ–ø—É—Å–∫–∞–µ–º —Å–ª–æ–≤–æ, –µ—Å–ª–∏ –Ω–µ —É–¥–∞–ª–æ—Å—å –æ–ø—Ä–µ–¥–µ–ª–∏—Ç—å —è–∑—ã–∫
        return foreign_word_count
    return 0

In [6]:
comments_df['Foreign_Words'] = comments_df['Processed_Text'].apply(count_foreign_words)

In [5]:

from deeppavlov import build_model, configs

model = build_model('levenshtein_corrector_ru', download=True)

2024-05-25 07:14:46.348 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz download because of matching hashes
2024-05-25 07:14:46.962 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz download because of matching hashes


In [49]:
i=899
print(comments_df['Processed_Text'][i])
model([comments_df['Processed_Text'][i]])

–æ—á–µ—Ä–µ–¥–Ω–æ–π –¥—Ä—è–Ω—å


['–æ—á–µ—Ä–µ–¥–Ω–æ–π –¥—Ä—è–Ω—å']

In [7]:
import pandas as pd
import spacy
import re
import string
from gensim.models import Word2Vec
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words

# –ó–∞–≥—Ä—É–∑–∫–∞ –º–æ–¥–µ–ª–∏ spaCy –¥–ª—è —Ä—É—Å—Å–∫–æ–≥–æ —è–∑—ã–∫–∞
nlp = spacy.load("ru_core_news_sm")
comments_df['Text'] = comments_df['Text'].fillna('').astype(str)
# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø—Ä–µ–¥–æ–±—Ä–∞–±–æ—Ç–∫–∏ —Ç–µ–∫—Å—Ç–∞
def preprocess_text(text):
    # –û–±—Ä–∞–±–æ—Ç–∫–∞ NaN –∑–Ω–∞—á–µ–Ω–∏–π
    if pd.isnull(text):
        return ""
    # –ü—Ä–∏–≤–µ–¥–µ–Ω–∏–µ –∫ –Ω–∏–∂–Ω–µ–º—É —Ä–µ–≥–∏—Å—Ç—Ä—É
    text = text.lower()
    # –£–¥–∞–ª–µ–Ω–∏–µ —Å—Å—ã–ª–æ–∫ –∏ –¥—Ä—É–≥–∏—Ö –Ω–µ–Ω—É–∂–Ω—ã—Ö —Å–∏–º–≤–æ–ª–æ–≤
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # –¢–æ–∫–µ–Ω–∏–∑–∞—Ü–∏—è –∏ –ª–µ–º–º–∞—Ç–∏–∑–∞—Ü–∏—è
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and token.is_alpha]
    return ' '.join(tokens)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ –ø—Ä–µ–¥–æ–±—Ä–∞–±–æ—Ç–∫–∏ –∫ –∫–∞–∂–¥–æ–º—É —Ç–µ–∫—Å—Ç—É
comments_df['Processed_Text'] = comments_df['Text'].apply(preprocess_text)

# –°–æ–∑–¥–∞–Ω–∏–µ –∫–æ—Ä–ø—É—Å–∞ –∏–∑ —Ç–µ–∫—Å—Ç–æ–≤ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤
corpus = comments_df['Processed_Text'].tolist()

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ —Ç–µ–∫—Å—Ç–∞ –Ω–∞ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏—è
sentences = [text.split() for text in corpus]

# –°–æ–∑–¥–∞–Ω–∏–µ –º–æ–¥–µ–ª–∏ Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_comment_vector(comment, model):
    words = comment.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# –ó–∞–≥—Ä—É–∑–∫–∞ –ø—Ä–µ–¥–æ–±—É—á–µ–Ω–Ω–æ–π –º–æ–¥–µ–ª–∏ Word2Vec
#with open('ruwiki_20180420_300d.txt', 'rb') as f:
#    pretrained_model = pickle.load(f)    

#pretrained_model = Word2Vec.load_word2vec_format('model.bin', binary=False)

# –ü—Ä–µ–æ–±—Ä–∞–∑–æ–≤–∞–Ω–∏–µ –≤—Å–µ—Ö –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤ –≤ –≤–µ–∫—Ç–æ—Ä—ã –∏ –¥–æ–±–∞–≤–ª–µ–Ω–∏–µ –≤ DataFrame
#comments_df['Vector_PT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, pretrained_model))



# –î–æ–æ–±—É—á–µ–Ω–∏–µ –ø—Ä–µ–¥–æ–±—É—á–µ–Ω–Ω–æ–π –º–æ–¥–µ–ª–∏ –Ω–∞ –≤–∞—à–∏—Ö –¥–∞–Ω–Ω—ã—Ö
#pretrained_model.build_vocab(sentences, update=True)
#pretrained_model.train(sentences, total_examples=pretrained_model.corpus_count, epochs=pretrained_model.epochs)
#comments_df['Vector_PT_FT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, pretrained_model))
# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø–æ–ª—É—á–µ–Ω–∏—è –≤–µ–∫—Ç–æ—Ä–∞ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è

    
def extract_additional_features(comment):
    length = len(comment)
    uppercase_count = sum(1 for c in comment if c.isupper())
    punctuation_count = sum(1 for c in comment if c in string.punctuation)
    emoji_count = len(re.findall(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+', comment))
    return np.array([length, uppercase_count, punctuation_count, emoji_count])

# –ü—Ä–µ–æ–±—Ä–∞–∑–æ–≤–∞–Ω–∏–µ –≤—Å–µ—Ö –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤ –≤ –≤–µ–∫—Ç–æ—Ä—ã –∏ –¥–æ–±–∞–≤–ª–µ–Ω–∏–µ –¥–æ–ø–æ–ª–Ω–∏—Ç–µ–ª—å–Ω—ã—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤
def get_combined_vector(comment, model):
    word_vector = get_comment_vector(comment, model)
    additional_features = extract_additional_features(comment)
    combined_vector = np.concatenate((word_vector, additional_features), axis=None)
    return combined_vector

# –ü—Ä–∏–º–µ—Ä –ø–æ–ª—É—á–µ–Ω–∏—è –≤–µ–∫—Ç–æ—Ä–∞ –¥–ª—è –æ–¥–Ω–æ–≥–æ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è
example_comment = comments_df['Processed_Text'].iloc[0]
comment_vector = get_comment_vector(example_comment, word2vec_model)
print(comment_vector)

# –ü—Ä–µ–æ–±—Ä–∞–∑–æ–≤–∞–Ω–∏–µ –≤—Å–µ—Ö –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤ –≤ –≤–µ–∫—Ç–æ—Ä—ã
comments_df['Vector'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, word2vec_model))

print(comments_df[['Text', 'Processed_Text', 'Vector']].head())


[-1.0654504e-03  1.0320158e-03  2.7321533e-03  1.4133225e-03
 -4.2301468e-03 -1.9832016e-03 -2.9613604e-03 -6.0447521e-04
 -8.1446749e-05  1.2731079e-03 -6.1814737e-04 -1.8282729e-03
  3.4352015e-03 -1.1375772e-03 -1.5653613e-03 -9.2165184e-04
  4.9283486e-03 -3.8856727e-03 -2.8179768e-03  2.2492285e-03
  1.7177633e-03  2.8284639e-04  1.8656129e-03 -8.8580657e-04
 -1.1668840e-04 -1.0964335e-03  2.6085475e-04  1.4139909e-03
 -3.5522154e-03  4.7996466e-04 -5.5389036e-04  4.3803913e-04
  6.9834321e-04  7.9575204e-04 -9.9613098e-06 -4.5002007e-04
 -1.0592618e-03 -1.8349282e-03  3.6572823e-03 -1.5381719e-03
  3.3216353e-04  1.5853625e-04  9.0166635e-04  4.2739804e-03
  1.3446755e-03 -1.2062155e-03  1.8733098e-04  2.1990722e-04
 -1.2826540e-03  1.5358465e-03 -5.7199667e-04  7.5791695e-04
 -9.7678741e-04  1.3685594e-03 -2.9003841e-04  5.1651604e-04
 -3.3517629e-03  1.0955312e-03  2.8434812e-04 -4.6028954e-04
 -2.3505653e-03  9.0783462e-04 -2.7266983e-03 -3.7410349e-04
 -2.3560040e-04 -1.16132

In [ ]:
import pandas as pd
import spacy
import re
import string
import numpy as np
from gensim.models import FastText
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words
import fasttext

# Load pre-trained FastText model for Russian
model_ft = fasttext.load_model('cc.ru.300.bin')

# –°–æ–∑–¥–∞–Ω–∏–µ –º–æ–¥–µ–ª–∏ FastText (–Ω–æ–≤–∞—è –º–æ–¥–µ–ª—å, –æ–±—É—á–µ–Ω–Ω–∞—è –Ω–∞ –≤–∞—à–∏—Ö –¥–∞–Ω–Ω—ã—Ö)
#fasttext_model = FastText(sentences, vector_size=300, window=5, min_count=1, workers=4)

# –ó–∞–≥—Ä—É–∑–∫–∞ –ø—Ä–µ–¥–æ–±—É—á–µ–Ω–Ω–æ–π –º–æ–¥–µ–ª–∏ FastText
#model_path = 'cc.ru.300.bin'  # –∑–∞–º–µ–Ω–∏—Ç–µ –Ω–∞ –ø—É—Ç—å –∫ –≤–∞—à–µ–π —Å–∫–∞—á–∞–Ω–Ω–æ–π –º–æ–¥–µ–ª–∏
#pretrained_model = FastText.load_facebook_model(model_path)
#pretrained_model = FastTexload_facebook_vectors(model_path)
# –î–æ–æ–±—É—á–µ–Ω–∏–µ –ø—Ä–µ–¥–æ–±—É—á–µ–Ω–Ω–æ–π –º–æ–¥–µ–ª–∏ –Ω–∞ –≤–∞—à–∏—Ö –¥–∞–Ω–Ω—ã—Ö
#pretrained_model.build_vocab(sentences, update=True)
#pretrained_model.train(sentences, total_examples=pretrained_model.corpus_count, epochs=pretrained_model.epochs)

# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø–æ–ª—É—á–µ–Ω–∏—è –≤–µ–∫—Ç–æ—Ä–∞ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è
def get_comment_vector(comment, model):
    if comment:
        print(len(model.get_sentence_vector(comment)))
        return model.get_sentence_vector(comment)
    else:
        return np.zeros(model.vector_size)

# –ü—Ä–µ–æ–±—Ä–∞–∑–æ–≤–∞–Ω–∏–µ –≤—Å–µ—Ö –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤ –≤ –≤–µ–∫—Ç–æ—Ä—ã –∏ –¥–æ–±–∞–≤–ª–µ–Ω–∏–µ –≤ DataFrame
comments_df['Vector_FT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, model_ft))
#comments_df['Vector_PT_FT'] = comments_df['Processed_Text'].apply(lambda x: get_comment_vector(x, pretrained_model))

# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø–æ–ª—É—á–µ–Ω–∏—è –¥–æ–ø–æ–ª–Ω–∏—Ç–µ–ª—å–Ω—ã—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤
def extract_additional_features(comment):
    length = len(comment)
    uppercase_count = sum(1 for c in comment if c.isupper())
    punctuation_count = sum(1 for c in comment if c in string.punctuation)
    emoji_count = len(re.findall(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+', comment))
    return np.array([length, uppercase_count, punctuation_count, emoji_count])


# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤
#print(comments_df[['Text', 'Processed_Text', 'Vector_PT']])


In [27]:
#comments_df.to_csv('vid_w2v.csv', index=False)

In [23]:
comments_df['Vector_features'] = comments_df['Text'].apply(lambda x: get_combined_vector(preprocess_text(x), word2vec_model))

In [ ]:
#comments_df['Vector'] = comments_df['Vector'].apply(lambda x: np.array(literal_eval(str(x))) if isinstance(x, str) else x)

In [40]:
#comments_df.to_csv('comments_f2v_fts.csv', index=False)

In [60]:
comm_features_label = pd.read_csv('vid_23.csv')

In [52]:
pip install autocorrect

     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 622.8/622.8 kB 1.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=a3e684b04efafa0fb29eee8d88f78b0dc880edaa2a2bc4e59bac53ad2103cd3e
  Stored in directory: /Users/pratappokkharel/Library/Caches/pip/wheels/5e/90/99/807a5ad861ce5d22c3c299a11df8cba9f31524f23ae6e645cb
Successfully built autocorrect
Note: you may need to restart the kernel to use updated packages.


In [34]:
from autocorrect import Speller
spell = Speller('ru')
i=100
print(comments_df['Processed_Text'][i])
spell(comments_df['Processed_Text'][i])


—Å–º–æ—Ç—Ä–µ—Ç—å –∏–Ω—Ç–µ—Ä–≤—å—é –≤–æ—Ä–æ—Ç–∏—Ç—å —Ä–æ—Å—Å–∏—è–Ω–∏–Ω —Ç–∏–ø–æ –ø—Ä–æ—Ç–∏–≤ –≤–æ–π–Ω–∞ –∑–∞–º–µ—á–∞—Ç—å –µ–º–æ—Ü–∏–π –≤—Ä–∞–∂–¥–∞ –ø–æ–∫–æ–ª–µ–Ω–∏–µ


'—Å–º–æ—Ç—Ä–µ—Ç—å –∏–Ω—Ç–µ—Ä–≤—å—é –≤–æ—Ä–æ—Ç–∏—Ç—å —Ä–æ—Å—Å–∏—è–Ω–∏–Ω —Ç–∏–ø–æ –ø—Ä–æ—Ç–∏–≤ –≤–æ–π–Ω–∞ –∑–∞–º–µ—á–∞—Ç—å —ç–º–æ—Ü–∏–π –≤—Ä–∞–∂–¥–∞ –ø–æ–∫–æ–ª–µ–Ω–∏–µ'

In [64]:
import nltk
from nltk.corpus import brown

# –ó–∞–≥—Ä—É–∂–∞–µ–º –∫–æ—Ä–ø—É—Å —Å–ª–æ–≤ –∏–∑ –±–∏–±–ª–∏–æ—Ç–µ–∫–∏ NLTK
nltk.download('brown')
nltk.download('words')
words = set(nltk.corpus.words.words())
brown_words = nltk.FreqDist(w.lower() for w in brown.words())

def is_valid_word(word):
    return word in words

def split_word(word):
    max_prob = 0
    best_split = None
    for i in range(1, len(word)):
        word1, word2 = word[:i], word[i:]
        if is_valid_word(word1) and is_valid_word(word2):
            # –ò—Å–ø–æ–ª—å–∑—É–µ–º —á–∞—Å—Ç–æ—Ç—ã –∏–∑ –∫–æ—Ä–ø—É—Å–∞ –¥–ª—è –æ—Ü–µ–Ω–∫–∏ –≤–µ—Ä–æ—è—Ç–Ω–æ—Å—Ç–∏
            prob = brown_words[word1] * brown_words[word2]
            if prob > max_prob:
                max_prob = prob
                best_split = (word1, word2)
    return best_split if best_split else (word,)

# –ü—Ä–∏–º–µ—Ä—ã –Ω–µ–ø—Ä–∞–≤–∏–ª—å–Ω—ã—Ö —Å–ª–æ–≤
incorrect_words = ['–º–∞–º–∞—Ä–∞–º–∞', '–∫–Ω–∏–∂–∫–∞—Å—Ç–æ–ª', '—Å—á–∞—Å—Ç—å–µ–ª—é–¥–µ–π']

# –ü–æ–∏—Å–∫ –∏ –ø–µ—á–∞—Ç—å —Ä–∞–∑–¥–µ–ª—ë–Ω–Ω—ã—Ö —Å–ª–æ–≤
for word in incorrect_words:
    split = split_word(word)
    print(f"{word} -> {' '.join(split)}")


[nltk_data] Downloading package brown to
[nltk_data]     /Users/pratappokkharel/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package words to
[nltk_data]     /Users/pratappokkharel/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


–º–∞–º–∞—Ä–∞–º–∞ -> –º–∞–º–∞—Ä–∞–º–∞
–∫–Ω–∏–∂–∫–∞—Å—Ç–æ–ª -> –∫–Ω–∏–∂–∫–∞—Å—Ç–æ–ª
—Å—á–∞—Å—Ç—å–µ–ª—é–¥–µ–π -> —Å—á–∞—Å—Ç—å–µ–ª—é–¥–µ–π


In [50]:
comments_df['Processed_Text'][15]

'—á–µ–ª–æ–≤–µ–∫ –±—ã–≤–∞—é—Ç–≤–∏—Ç–∞–µ—Ç –æ–±–ª–∞–∫–æ –Ω–µ–ø–ª–æ—Ö–æ–Ω–æ –ø–æ–Ω–∏–º–∞—Ç—å —Ä–µ–∞–ª—å–Ω–æ—Å—Ç—å —Ç–∞–º–Ω–µ–ª—å–∑—è –ø–µ—Ä–µ–µ—Ö–∞—Ç—å –±—ã—Å—Ç—Ä–æ –≤–∫–ª—é—á–∏—Ç—å—Å—è'

In [28]:
pip install pymorphy2

  Obtaining dependency information for pymorphy2 from https://files.pythonhosted.org/packages/07/57/b2ff2fae3376d4f3c697b9886b64a54b476e1a332c67eee9f88e7f1ae8c9/pymorphy2-0.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for pymorphy2-dicts-ru<3.0,>=2.4 from https://files.pythonhosted.org/packages/3a/79/bea0021eeb7eeefde22ef9e96badf174068a2dd20264b9a378f2be1cdd9e/pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl.metadata
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=2c48a3383c8984ab9c30828450e53fbbc2c6cbea3ee66563a15db49f13f6fc70
  Stored in directory: /Users/pratappokkharel/Library/Caches/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
Note: you may need

In [35]:
import nltk
import pymorphy2
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams


# –ó–∞–≥—Ä—É–∂–∞–µ–º —Å–ª–æ–≤–∞
with open('100000-russian-words.txt', 'r', encoding='utf-8') as f:
    correct_words = f.read().split()

# –ü—Ä–∏–º–µ—Ä—ã –Ω–µ–ø—Ä–∞–≤–∏–ª—å–Ω—ã—Ö —Å–ª–æ–≤
incorrect_words = ['—Å—á–∞—Å—Ç—Ç–ª–∏–≤—ã–π', '—É–¥–µ–≤–∏—Ç–µ–ª—å–Ω–æ–µ', '–∏–Ω—Ç–µ–ª–ª–µ–≥–µ–Ω—Ç–Ω—ã–π']

# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –∏—Å–ø—Ä–∞–≤–ª–µ–Ω–∏—è —Å–ª–æ–≤
def correct_spelling(word, correct_words):
    morph = pymorphy2.MorphAnalyzer()
    temp = [(jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))), w) 
            for w in correct_words if w[0] == word[0]]
    return sorted(temp, key=lambda val: val[0])[0][1] if temp else word

# –ü–æ–∏—Å–∫ –∏ –ø–µ—á–∞—Ç—å –∏—Å–ø—Ä–∞–≤–ª–µ–Ω–Ω—ã—Ö —Å–ª–æ–≤
for word in incorrect_words:
    corrected_word = correct_spelling(word, correct_words)
    print(corrected_word)

    print(corrected_word)



AttributeError: module 'inspect' has no attribute 'getargspec'

In [64]:
comm_features_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1000 non-null   int64  
 1   author_name               999 non-null    object 
 2   text                      1000 non-null   object 
 3   reply_count               1000 non-null   float64
 4   top_level                 1000 non-null   int64  
 5   index                     1000 non-null   int64  
 6   publishedAt               1000 non-null   object 
 7   updateAt                  1000 non-null   object 
 8   likeCount                 1000 non-null   float64
 9   Video_ID                  1000 non-null   int64  
 10  Label                     1000 non-null   object 
 11  Text                      995 non-null    object 
 12  Length                    1000 non-null   int64  
 13  UpperCase                 1000 non-null   int64  
 14  Punctuati

In [61]:
comments_df_fin = comments_df.iloc[:1000]

In [62]:
comments_df_fin['Label'] = comm_features_label['Label']

/var/folders/00/4_gxgzyx7131tsyx_ymz0hsm0000gn/T/ipykernel_1945/3002675891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df_fin['Label'] = comm_features_label['Label']


# Word2vec

In [66]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train, y_train)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.92
F1 Score (Micro): 0.92
F1 Score (Macro): 0.46208112874779544
ROC AUC (OvO): 0.6558157219469972
ROC AUC (OvR): 0.6558157219469972
              precision    recall  f1-score   support

         bot       1.00      0.27      0.43        11
      normal       0.92      1.00      0.96       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.92       200
   macro avg       0.64      0.42      0.46       200
weighted avg       0.89      0.92      0.89       200


Logistic Regression:
Accuracy: 0.905
F1 Score (Micro): 0.905
F1 Score (Macro): 0.31671041119860016
ROC AUC (OvO): 0.6053368468977264
ROC AUC (OvR): 0.6053368468977264
              precision    recall  f1-score   support

         bot       0.00      0.00      0.00        11
      normal       0.91      1.00      0.95       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.91       200
   macro avg       0

/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pratappokkharel/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

# Word2vec + SMOTE

In [67]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.795
F1 Score (Micro): 0.795
F1 Score (Macro): 0.39221304567672344
ROC AUC (OvO): 0.7197904421437938
ROC AUC (OvR): 0.7197904421437938
              precision    recall  f1-score   support

         bot       0.22      0.45      0.29        11
      normal       0.92      0.85      0.88       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.80       200
   macro avg       0.38      0.44      0.39       200
weighted avg       0.84      0.80      0.81       200


Logistic Regression:
Accuracy: 0.275
F1 Score (Micro): 0.275
F1 Score (Macro): 0.23045065220807792
ROC AUC (OvO): 0.6539688589267714
ROC AUC (OvR): 0.6539688589267714
              precision    recall  f1-score   support

         bot       0.08      0.64      0.14        11
      normal       0.95      0.23      0.37       181
       troll       0.10      0.88      0.19         8

    accuracy                           0.28       200
   macro avg      

# Standardized Word2vec + SMOTE + 

In [68]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# –°—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏—è –ø—Ä–∏–∑–Ω–∞–∫–æ–≤
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test = scaler.transform(X_test)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.79
F1 Score (Micro): 0.79
F1 Score (Macro): 0.3883415435139573
ROC AUC (OvO): 0.7183169549747657
ROC AUC (OvR): 0.7183169549747657
              precision    recall  f1-score   support

         bot       0.21      0.45      0.29        11
      normal       0.92      0.85      0.88       181
       troll       0.00      0.00      0.00         8

    accuracy                           0.79       200
   macro avg       0.37      0.43      0.39       200
weighted avg       0.84      0.79      0.81       200


Logistic Regression:
Accuracy: 0.52
F1 Score (Micro): 0.52
F1 Score (Macro): 0.34145880574452003
ROC AUC (OvO): 0.6687594327671461
ROC AUC (OvR): 0.6687594327671461
              precision    recall  f1-score   support

         bot       0.11      0.64      0.19        11
      normal       0.95      0.51      0.67       181
       troll       0.10      0.50      0.16         8

    accuracy                           0.52       200
   macro avg       0.39

# Word2vec + SMOTE + Features

In [69]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.64
F1 Score (Micro): 0.64
F1 Score (Macro): 0.36143237605691514
ROC AUC (OvO): 0.686852599321098
ROC AUC (OvR): 0.686852599321098
              precision    recall  f1-score   support

         bot       0.14      0.45      0.21        11
      normal       0.91      0.67      0.77       181
       troll       0.07      0.25      0.11         8

    accuracy                           0.64       200
   macro avg       0.37      0.46      0.36       200
weighted avg       0.83      0.64      0.71       200


Logistic Regression:
Accuracy: 0.26
F1 Score (Micro): 0.26
F1 Score (Macro): 0.22210820446114565
ROC AUC (OvO): 0.6332451479631573
ROC AUC (OvR): 0.6332451479631573
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       0.95      0.22      0.35       181
       troll       0.12      0.88      0.21         8

    accuracy                           0.26       200
   macro avg       0.38 

# Word2vec + SMOTE + Standardized Features


In [70]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# –í—ã–¥–µ–ª–µ–Ω–∏–µ –ø–æ—Å–ª–µ–¥–Ω–∏—Ö —á–µ—Ç—ã—Ä–µ—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤ –¥–ª—è —Å—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏–∏
X_additional = X[:, -4:]

# –°—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏—è –ø–æ—Å–ª–µ–¥–Ω–∏—Ö —á–µ—Ç—ã—Ä–µ—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤
scaler = StandardScaler()
X_additional_scaled = scaler.fit_transform(X_additional)

# –û–±—ä–µ–¥–∏–Ω–µ–Ω–∏–µ —Å—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∏—Ä–æ–≤–∞–Ω–Ω—ã—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤ —Å –æ—Å—Ç–∞–ª—å–Ω—ã–º–∏
X_combined = np.hstack((X[:, :-4], X_additional_scaled))

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.775
F1 Score (Micro): 0.775
F1 Score (Macro): 0.44004121423476267
ROC AUC (OvO): 0.6904555357813633
ROC AUC (OvR): 0.6904555357813633
              precision    recall  f1-score   support

         bot       0.19      0.45      0.27        11
      normal       0.93      0.82      0.87       181
       troll       0.14      0.25      0.18         8

    accuracy                           0.78       200
   macro avg       0.42      0.51      0.44       200
weighted avg       0.85      0.78      0.81       200


Logistic Regression:
Accuracy: 0.25
F1 Score (Micro): 0.25
F1 Score (Macro): 0.2155307994757536
ROC AUC (OvO): 0.6245253183651656
ROC AUC (OvR): 0.6245253183651656
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       1.00      0.20      0.34       181
       troll       0.11      0.88      0.20         8

    accuracy                           0.25       200
   macro avg       0.

# PCA (Word2vec) + SMOTE

In [73]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.decomposition import PCA

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector'].values)
y = comments_df_fin['Label']

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ PCA –¥–ª—è —É–º–µ–Ω—å—à–µ–Ω–∏—è —Ä–∞–∑–º–µ—Ä–Ω–æ—Å—Ç–∏ –≤–µ–∫—Ç–æ—Ä–æ–≤ Word2Vec
pca = PCA(n_components=25, random_state=42)
X_pca = pca.fit_transform(X)

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42, kernel='rbf')
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.865
F1 Score (Micro): 0.865
F1 Score (Macro): 0.5404007756948933
ROC AUC (OvO): 0.8290817200955655
ROC AUC (OvR): 0.8290817200955655
              precision    recall  f1-score   support

         bot       0.40      0.55      0.46        11
      normal       0.94      0.91      0.92       181
       troll       0.22      0.25      0.24         8

    accuracy                           0.86       200
   macro avg       0.52      0.57      0.54       200
weighted avg       0.88      0.86      0.87       200


Logistic Regression:
Accuracy: 0.29
F1 Score (Micro): 0.29
F1 Score (Macro): 0.23942284294707203
ROC AUC (OvO): 0.6525923527472473
ROC AUC (OvR): 0.6525923527472473
              precision    recall  f1-score   support

         bot       0.08      0.64      0.14        11
      normal       0.96      0.24      0.39       181
       troll       0.11      0.88      0.19         8

    accuracy                           0.29       200
   macro avg       0.

In [40]:
# PCA (Standardized Word2vec) + SMOTE

In [60]:
comments_df_fin['Vector_features'][0].size

104

# PCA (Word2vec) + SMOTE + Features

In [74]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector_features'].values)
y = comments_df_fin['Label']

# –í—ã–¥–µ–ª–µ–Ω–∏–µ –ø–æ—Å–ª–µ–¥–Ω–∏—Ö —á–µ—Ç—ã—Ä–µ—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤ –¥–ª—è —Å—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏–∏
X_additional = X[:, -4:]

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ PCA –¥–ª—è —É–º–µ–Ω—å—à–µ–Ω–∏—è —Ä–∞–∑–º–µ—Ä–Ω–æ—Å—Ç–∏ –≤–µ–∫—Ç–æ—Ä–æ–≤ Word2Vec
pca = PCA(n_components=25, random_state=42)
X_pca = pca.fit_transform(X[:, :-4])

# –°—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏—è –ø–æ—Å–ª–µ–¥–Ω–∏—Ö —á–µ—Ç—ã—Ä–µ—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤
#scaler = StandardScaler()
#X_additional_scaled = scaler.fit_transform(X_additional)

# –û–±—ä–µ–¥–∏–Ω–µ–Ω–∏–µ —Å—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∏—Ä–æ–≤–∞–Ω–Ω—ã—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤ —Å –æ—Å—Ç–∞–ª—å–Ω—ã–º–∏
X_combined = np.hstack((X_pca, X_additional))

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# –°–æ–∑–¥–∞–Ω–∏–µ –∏ –æ–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model = SVC(probability=True, random_state=42)
nb_model = MultinomialNB()

models = {
    "Random Forest": rf_model,
    "Logistic Regression": lr_model,
    "Gradient Boosting": gb_model,
    "SVC": svc_model,
#    "Naive Bayes": nb_model
}

# –û–±—É—á–µ–Ω–∏–µ –º–æ–¥–µ–ª–µ–π
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ –º–æ–¥–µ–ª–µ–π
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –í—ã–≤–æ–¥ —Ä–µ–∑—É–ª—å—Ç–∞—Ç–æ–≤ –¥–ª—è –∫–∞–∂–¥–æ–π –º–æ–¥–µ–ª–∏
for name, model in models.items():
    y_pred = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # For models without predict_proba (e.g., SVC without probability=True)
        ovo_classifier = OneVsOneClassifier(model)
        ovo_classifier.fit(X_train, y_train)
        y_proba = ovo_classifier.decision_function(X_test)
    evaluate_model(y_test, y_pred, y_proba, name)

Random Forest:
Accuracy: 0.875
F1 Score (Micro): 0.875
F1 Score (Macro): 0.5481664517977367
ROC AUC (OvO): 0.7794330055545049
ROC AUC (OvR): 0.7794330055545049
              precision    recall  f1-score   support

         bot       0.40      0.55      0.46        11
      normal       0.94      0.92      0.93       181
       troll       0.25      0.25      0.25         8

    accuracy                           0.88       200
   macro avg       0.53      0.57      0.55       200
weighted avg       0.89      0.88      0.88       200


Logistic Regression:
Accuracy: 0.265
F1 Score (Micro): 0.265
F1 Score (Macro): 0.22559684253011336
ROC AUC (OvO): 0.634764792540074
ROC AUC (OvR): 0.634764792540074
              precision    recall  f1-score   support

         bot       0.06      0.55      0.11        11
      normal       0.95      0.22      0.36       181
       troll       0.12      0.88      0.21         8

    accuracy                           0.27       200
   macro avg       0.

In [58]:
# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –¥–∞–Ω–Ω—ã—Ö –Ω–∞ –ø—Ä–∏–∑–Ω–∞–∫–∏ –∏ —Ü–µ–ª–µ–≤—É—é –ø–µ—Ä–µ–º–µ–Ω–Ω—É—é
X = np.vstack(comments_df_fin['Vector_features'].values)
X_word2vec = X[:, :-4]
# –ó–¥–µ—Å—å –º–æ–∂–Ω–æ –¥–æ–±–∞–≤–∏—Ç—å —Å–≤–æ–∏ –ø—Ä–∏–∑–Ω–∞–∫–∏
# –í—ã–¥–µ–ª–µ–Ω–∏–µ –ø–æ—Å–ª–µ–¥–Ω–∏—Ö —á–µ—Ç—ã—Ä–µ—Ö –ø—Ä–∏–∑–Ω–∞–∫–æ–≤ –¥–ª—è —Å—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏–∏
X_additional = X[:, -4:]
y = comments_df_fin['Label']

# –°—Ç–∞–Ω–¥–∞—Ä—Ç–∏–∑–∞—Ü–∏—è –¥–∞–Ω–Ω—ã—Ö
#scaler_word2vec = StandardScaler()
#X_word2vec = scaler_word2vec.fit_transform(X_word2vec)

scaler_additional = StandardScaler()
X_additional = scaler_additional.fit_transform(X_additional)

n_components = 25
# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ PCA —Å –≤—ã–±—Ä–∞–Ω–Ω—ã–º —á–∏—Å–ª–æ–º –∫–æ–º–ø–æ–Ω–µ–Ω—Ç–æ–≤
pca = PCA(n_components=n_components, random_state=42)
X_word2vec_pca = pca.fit_transform(X_word2vec)

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏
X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec_pca, y, test_size=0.2, random_state=42)
X_train_additional, X_test_additional, _, _ = train_test_split(X_additional, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ
smote = SMOTE(random_state=42)
X_train_word2vec_smote, y_train_smote = smote.fit_resample(X_train_word2vec, y_train)
X_train_additional_smote, _ = smote.fit_resample(X_train_additional, y_train)

# –°–æ–∑–¥–∞–Ω–∏–µ –º–æ–¥–µ–ª–µ–π –¥–ª—è –∫–∞–∂–¥–æ–π –≥—Ä—É–ø–ø—ã –ø—Ä–∏–∑–Ω–∞–∫–æ–≤
rf_model_word2vec = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model_word2vec = LogisticRegression(max_iter=1000, random_state=42)
gb_model_word2vec = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model_word2vec = SVC(probability=True, random_state=42)

rf_model_additional = RandomForestClassifier(n_estimators=100, random_state=42)
lr_model_additional = LogisticRegression(max_iter=1000, random_state=42)
gb_model_additional = GradientBoostingClassifier(n_estimators=100, random_state=42)
svc_model_additional = SVC(probability=True, random_state=42)

# –°–æ–∑–¥–∞–Ω–∏–µ —Å–ª–æ–≤–∞—Ä—è –±–∞–∑–æ–≤—ã—Ö –º–æ–¥–µ–ª–µ–π
base_models_word2vec = {
    "Random Forest Word2Vec": rf_model_word2vec,
    #"Logistic Regression Word2Vec": lr_model_word2vec,
    "Gradient Boosting Word2Vec": gb_model_word2vec,
    #"SVC Word2Vec": svc_model_word2vec
}

base_models_additional = {
    #"Random Forest Additional": rf_model_additional,
    #"Logistic Regression Additional": lr_model_additional,
    #"Gradient Boosting Additional": gb_model_additional,
    "SVC Additional": svc_model_additional
}

# –ü–æ–ª—É—á–µ–Ω–∏–µ –º–µ—Ç–∞–ø—Ä–∏–∑–Ω–∞–∫–æ–≤ —á–µ—Ä–µ–∑ –∫—Ä–æ—Å—Å-–≤–∞–ª–∏–¥–∞—Ü–∏—é
def get_meta_features(model_dict, X, y):
    meta_features = np.zeros((X.shape[0], len(model_dict)))
    for i, (name, model) in enumerate(model_dict.items()):
        meta_features[:, i] = cross_val_predict(model, X, y, cv=5, method='predict_proba')[:, 1]
    return meta_features

meta_features_word2vec = get_meta_features(base_models_word2vec, X_word2vec_pca, y)
meta_features_additional = get_meta_features(base_models_additional, X_additional, y)

# –û–±—ä–µ–¥–∏–Ω–µ–Ω–∏–µ –º–µ—Ç–∞–ø—Ä–∏–∑–Ω–∞–∫–æ–≤
meta_features_combined = np.hstack([meta_features_word2vec, meta_features_additional])

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ –Ω–∞ –æ–±—É—á–∞—é—â—É—é –∏ —Ç–µ—Å—Ç–æ–≤—É—é –≤—ã–±–æ—Ä–∫–∏ –¥–ª—è –º–µ—Ç–∞–º–æ–¥–µ–ª–∏
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(meta_features_combined, y, test_size=0.2, random_state=42)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ SMOTE –∫ –æ–±—É—á–∞—é—â–µ–π –≤—ã–±–æ—Ä–∫–µ –º–µ—Ç–∞–º–æ–¥–µ–ª–∏
X_train_meta_smote, y_train_meta_smote = smote.fit_resample(X_train_meta, y_train_meta)

from sklearn.ensemble import GradientBoostingClassifier

# –û–±—É—á–µ–Ω–∏–µ Logistic Regression –∫–∞–∫ –º–µ—Ç–∞–º–æ–¥–µ–ª–∏
lr_meta_model = LogisticRegression(max_iter=1000, random_state=42)
lr_meta_model.fit(X_train_meta_smote, y_train_meta_smote)

# –û–±—É—á–µ–Ω–∏–µ Gradient Boosting –∫–∞–∫ –º–µ—Ç–∞–º–æ–¥–µ–ª–∏
gb_meta_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_meta_model.fit(X_train_meta, y_train_meta)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ Logistic Regression –º–µ—Ç–∞–º–æ–¥–µ–ª–∏
y_pred_lr_meta = lr_meta_model.predict(X_test_meta)
y_proba_lr_meta = lr_meta_model.predict_proba(X_test_meta)

# –ü—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–µ –Ω–∞ —Ç–µ—Å—Ç–æ–≤—ã—Ö –¥–∞–Ω–Ω—ã—Ö –∏ –æ—Ü–µ–Ω–∫–∞ Gradient Boosting –º–µ—Ç–∞–º–æ–¥–µ–ª–∏
y_pred_gb_meta = gb_meta_model.predict(X_test_meta)
y_proba_gb_meta = gb_meta_model.predict_proba(X_test_meta)

# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –æ—Ü–µ–Ω–∫–∏ –º–æ–¥–µ–ª–∏
def evaluate_model(y_test, y_pred, y_proba, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_binarized = lb.transform(y_test)
    roc_auc_ovo = roc_auc_score(y_test_binarized, y_proba, multi_class='ovo')
    roc_auc_ovr = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
    print(f"{model_name}:")
    print("Accuracy:", accuracy)
    print("F1 Score (Micro):", f1_micro)
    print("F1 Score (Macro):", f1_macro)
    print("ROC AUC (OvO):", roc_auc_ovo)
    print("ROC AUC (OvR):", roc_auc_ovr)
    print(classification_report(y_test, y_pred))
    print()

# –û—Ü–µ–Ω–∫–∞ –º–µ—Ç–∞–º–æ–¥–µ–ª–µ–π
evaluate_model(y_test_meta, y_pred_lr_meta, y_proba_lr_meta, "Logistic Regression Meta Model")
evaluate_model(y_test_meta, y_pred_gb_meta, y_proba_gb_meta, "Gradient Boosting Meta Model")


Logistic Regression Meta Model:
Accuracy: 0.2830188679245283
F1 Score (Micro): 0.2830188679245283
F1 Score (Macro): 0.23836657169990505
ROC AUC (OvO): 0.44903846153846155
ROC AUC (OvR): 0.44903846153846155
              precision    recall  f1-score   support

         bot       0.10      0.40      0.15         5
      normal       0.79      0.28      0.41        40
       troll       0.11      0.25      0.15         8

    accuracy                           0.28        53
   macro avg       0.33      0.31      0.24        53
weighted avg       0.62      0.28      0.35        53


Gradient Boosting Meta Model:
Accuracy: 0.6981132075471698
F1 Score (Micro): 0.6981132075471698
F1 Score (Macro): 0.4102067183462533
ROC AUC (OvO): 0.491951566951567
ROC AUC (OvR): 0.491951566951567
              precision    recall  f1-score   support

         bot       0.33      0.20      0.25         5
      normal       0.76      0.88      0.81        40
       troll       0.25      0.12      0.17       

In [23]:
def preprocess_text(text):
    text = text.lower()  # –ü—Ä–∏–≤–µ–¥–µ–Ω–∏–µ —Ç–µ–∫—Å—Ç–∞ –∫ –Ω–∏–∂–Ω–µ–º—É —Ä–µ–≥–∏—Å—Ç—Ä—É
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # –£–¥–∞–ª–µ–Ω–∏–µ —Å—Å—ã–ª–æ–∫
    text = re.sub(r'\@\w+|\#', '', text)  # –£–¥–∞–ª–µ–Ω–∏–µ —É–ø–æ–º–∏–Ω–∞–Ω–∏–π –∏ —Ö—ç—à—Ç–µ–≥–æ–≤
    text = re.sub(r'\d+', '', text)  # –£–¥–∞–ª–µ–Ω–∏–µ —Ü–∏—Ñ—Ä
    text = text.translate(str.maketrans('', '', string.punctuation))  # –£–¥–∞–ª–µ–Ω–∏–µ –ø—É–Ω–∫—Ç—É–∞—Ü–∏–∏
    return text

In [25]:
# –°–æ–∑–¥–∞–Ω–∏–µ –º–æ–¥–µ–ª–∏ Word2Vec 
# –û–±—ä–µ–¥–∏–Ω–µ–Ω–∏–µ —Ç–µ–∫—Å—Ç–æ–≤ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤ –∏ –≤–∏–¥–µ–æ –≤ –µ–¥–∏–Ω—ã–π –∫–æ—Ä–ø—É—Å 
corpus = comments_df['Text'].apply(preprocess_text).tolist() #+ videos_df['Video_Text'].apply(preprocess_text).tolist() 
print(corpus)
sentences = [text.split() for text in corpus] 
 
# –°–æ–∑–¥–∞–Ω–∏–µ –º–æ–¥–µ–ª–∏ Word2Vec 
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4) 
 
# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø–æ–ª—É—á–µ–Ω–∏—è –≤–µ–∫—Ç–æ—Ä–∞ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è 
def get_comment_vector(comment, model): 
    words = comment.split() 
    word_vectors = [model.wv[word] for word in words if word in model.wv] 
    if len(word_vectors) == 0: 
        return np.zeros(model.vector_size) 
    else: 
        return np.mean(word_vectors, axis=0)

['–∫–ª–∞—Å—Å–∏—á–µ—Å–∫–∞—è –∏–Ω—Ñ–∞–Ω—Ç–∏–ª—å–Ω–∞—è —Ç–ø –ø—Ä–∏–¥—É–º–∞–ª–∞ —Å–µ–±–µ –≥–æ–ª–æ–≤–Ω—è–∫ –Ω–∞ —Ä–æ–≤–Ω–æ–º –º–µ—Å—Ç–µ —É–º–æ—Ç–∞–ª–∞ —Ç—É–¥–∞ –≥–¥–µ —Ç–µ–ø–ª–µ–µ', '–∑–∞—à–∫–≤–∞—Ä', '–Ω–∞—Ä–µ–∂—å—Ç–µ –≤–∏–¥–µ–æ —Å–æ –≤—Å–µ–º–∏ –µ–µ –º–æ–∫–∞–Ω–∏—è–º–∏ –ø–ª–∏–∑', '–µ—Å–ª–∏ –æ–Ω–∞ –µ—â–µ —Ä–∞–∑ —á–º–æ–∫–Ω–µ—Ç —è –æ–≥–ª–æ—Ö–Ω—É –Ω–∞–≤—Å–µ–≥–¥–∞\r\n–±—É–¥—Ç–æ –ø–æ–º–∏–¥–æ—Ä—ã –∫–≤–∞—à–µ–Ω—ã–µ –µ—Å—Ç—å', 'smm jump  –ª—É—á—à–∞—è –Ω–∞–∫—Ä—É—Ç–∫–∞ –≤ –∏–Ω—Ç–µ—Ä–Ω–µ—Ç–µ', '–∏–Ω—Å—Ç—Ä—É–∫—Ü–∏—è —è—à–∏–Ω–∞ –æ —Ç–æ–º –∫–∞–∫ –∫–≤–∞—Ä—Ç–∏—Ä—É –ø—Ä–∏–≥–æ—Ç–æ–≤–∏—Ç—å –∫ –æ–±—ã—Å–∫—É —Å–ª–µ–∑—Ç—å —Å —Å–æ–±—á–∞–∫ –∏ –Ω–∞–¥–µ—Ç—å —Ç—Ä—É—Å—ã', '–ø—Ä–æ –∫–æ—Ä–µ—é –≤—ã–ø—É—Å–∫ –≤–µ–ª–∏–∫–æ–ª–µ–ø–Ω—ã–π –∫–∞–∫ —Ä–∞–Ω—å—à–µ —Å–µ–π—á–∞—Å –æ–ø—è—Ç—å –ø–æ—à–ª–∞ –Ω—É–¥—è—Ç–∏–Ω–∞ —Å –±–µ—Å–∫–æ–Ω–µ—á–Ω—ã–º–∏ —Å—Ç–µ–Ω–¥–∞–ø–µ—Ä–∞–º–∏ –∏ –Ω–µ–ø–æ–Ω—è—Ç–Ω—ã–º–∏ –∞–∫—Ç—Ä–∏—Å–∫–∞–º–∏ —Ç–∞–∫–∂–µ –ø–∏—à—É—Ç—Å—è —Ñ–µ–º–∏–Ω–∏—Ç–∏–≤—ã\r\n–Ω–µ –ø–æ–Ω–∏–º–∞—é –ø–æ—á–µ–º—É –æ–± —ç—Ç–æ–º –Ω–∏ –∫—Ç–æ –Ω–µ –ø–∏—à–µ—Ç –≤

NameError: name 'Word2Vec' is not defined

In [16]:
import spacy
from gensim.utils import simple_preprocess
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words

# –ó–∞–≥—Ä—É–∑–∫–∞ –º–æ–¥–µ–ª–∏ spaCy –¥–ª—è —Ä—É—Å—Å–∫–æ–≥–æ —è–∑—ã–∫–∞
nlp = spacy.load("ru_core_news_sm")

def lemma(word):
    for w in nlp(word):
        return w.lemma_

def process_words(texts):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts_out = []
    for sent in texts:
        texts_out.append([lemma(token) for token in sent])
    # remove stopwords after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

def preprocess(comment):
    # Tokenize and lemmatize
    tokens = [token.lemma_ for token in nlp(comment)]
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens).replace('\n', ' ')

# –ü—Ä–∏–º–µ—Ä –∏—Å–ø–æ–ª—å–∑–æ–≤–∞–Ω–∏—è
texts = [
    "–≠—Ç–æ –ø—Ä–∏–º–µ—Ä —Ç–µ–∫—Å—Ç–∞ –¥–ª—è –æ–±—Ä–∞–±–æ—Ç–∫–∏. –í–∫–ª—é—á–∞–µ—Ç —Å—Ç–æ–ø-—Å–ª–æ–≤–∞ –∏ —Ä–∞–∑–ª–∏—á–Ω—ã–µ —Ñ–æ—Ä–º—ã —Å–ª–æ–≤.",
    "–ï—â–µ –æ–¥–∏–Ω –ø—Ä–∏–º–µ—Ä —Ç–µ–∫—Å—Ç–∞, –∫–æ—Ç–æ—Ä—ã–π –±—É–¥–µ—Ç –ª–µ–º–º–∞—Ç–∏–∑–∏—Ä–æ–≤–∞–Ω –∏ –æ—á–∏—â–µ–Ω –æ—Ç —Å—Ç–æ–ø-—Å–ª–æ–≤."
]

# –ü—Ä–∏–º–µ—Ä –ø—Ä–µ–¥–æ–±—Ä–∞–±–æ—Ç–∫–∏ —Ç–µ–∫—Å—Ç–æ–≤
processed_texts = process_words(texts)
print(processed_texts)

# –ü—Ä–∏–º–µ—Ä –ø—Ä–µ–¥–æ–±—Ä–∞–±–æ—Ç–∫–∏ –æ–¥–Ω–æ–≥–æ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è
comment = "–≠—Ç–æ –ø—Ä–∏–º–µ—Ä –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è –¥–ª—è –ª–µ–º–º–∞—Ç–∏–∑–∞—Ü–∏–∏ –∏ —É–¥–∞–ª–µ–Ω–∏—è —Å—Ç–æ–ø-—Å–ª–æ–≤."
processed_comment = preprocess(comment)
print(processed_comment)

[['–ø—Ä–∏–º–µ—Ä', '—Ç–µ–∫—Å—Ç', '–æ–±—Ä–∞–±–æ—Ç–∫–∞', '–≤–∫–ª—é—á–∞—Ç—å', '—Å—Ç–æ–ø', '—Å–ª–æ–≤–æ', '—Ä–∞–∑–ª–∏—á–Ω—ã–π', '—Ñ–æ—Ä–º–∞', '—Å–ª–æ–≤–æ'], ['–ø—Ä–∏–º–µ—Ä', '—Ç–µ–∫—Å—Ç', '–ª–µ–º–º–∞—Ç–∏–∑–∏—Ä–æ–≤–∞–Ω', '–æ—á–∏—Å—Ç–∏—Ç—å', '—Å—Ç–æ–ø', '—Å–ª–æ–≤–æ']]
–ø—Ä–∏–º–µ—Ä –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–π –ª–µ–º–º–∞—Ç–∏–∑–∞—Ü–∏—è —É–¥–∞–ª–µ–Ω–∏–µ —Å—Ç–æ–ø–∞ - —Å–ª–æ–≤–æ .


In [ ]:
import pandas as pd
import spacy
from gensim.utils import simple_preprocess
from spacy.lang.ru.stop_words import STOP_WORDS as stop_words

# –ó–∞–≥—Ä—É–∑–∫–∞ –º–æ–¥–µ–ª–∏ spaCy –¥–ª—è —Ä—É—Å—Å–∫–æ–≥–æ —è–∑—ã–∫–∞
nlp = spacy.load("ru_core_news_sm")

# –ü—Ä–∏–º–µ—Ä —Ç–µ–∫—Å—Ç–∞ –¥–ª—è –æ–±—Ä–∞–±–æ—Ç–∫–∏
texts = ['–∞ –º—ã —ç—Ç–æ –∫—Ç–æ —Ç—ã –∏ —Ç–≤–æ—è —à–∏–∑–∞üôÇ', '–º—ã –∫—Ä–æ–º–µ —É—Ä–∞–Ω–æ—Å–∞', '—ç—Ç–æ –∫—Ç–æ', '–æ–π–∫–∞–∫ —Ö–æ—Ä–æ—à–æ –Ω–∞ –∑–∞–ø–∞–¥–µ–Ω–æ –≤—Å—è –±–µ–¥–∞ –≤ —Å–∞–º–æ–π –∏ –≤–∏–∂—É —Å–ª–∞–±–æ–≥–æ —Ä—è–¥–æ–º –º—É–∂—á–∏–Ω—É', ...]

# –§—É–Ω–∫—Ü–∏—è –ª–µ–º–º–∞—Ç–∏–∑–∞—Ü–∏–∏ –∏ –ø—Ä–µ–¥–æ–±—Ä–∞–±–æ—Ç–∫–∏ —Ç–µ–∫—Å—Ç–∞
def preprocess_text(text):
    # –û–±—Ä–∞–±–æ—Ç–∫–∞ NaN –∑–Ω–∞—á–µ–Ω–∏–π
    if pd.isnull(text):
        return ""
    # –ü—Ä–∏–≤–µ–¥–µ–Ω–∏–µ –∫ –Ω–∏–∂–Ω–µ–º—É —Ä–µ–≥–∏—Å—Ç—Ä—É
    text = text.lower()
    # –£–¥–∞–ª–µ–Ω–∏–µ —Å—Å—ã–ª–æ–∫ –∏ –¥—Ä—É–≥–∏—Ö –Ω–µ–Ω—É–∂–Ω—ã—Ö —Å–∏–º–≤–æ–ª–æ–≤
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # –¢–æ–∫–µ–Ω–∏–∑–∞—Ü–∏—è –∏ –ª–µ–º–º–∞—Ç–∏–∑–∞—Ü–∏—è
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and token.is_alpha]
    return ' '.join(tokens)

# –ü—Ä–∏–º–µ–Ω–µ–Ω–∏–µ –ø—Ä–µ–¥–æ–±—Ä–∞–±–æ—Ç–∫–∏ –∫ –∫–∞–∂–¥–æ–º—É —Ç–µ–∫—Å—Ç—É
processed_texts = [preprocess_text(text) for text in texts]

print(processed_texts)

# –°–æ–∑–¥–∞–Ω–∏–µ –∫–æ—Ä–ø—É—Å–∞ –∏–∑ —Ç–µ–∫—Å—Ç–æ–≤ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏–µ–≤
corpus = processed_texts

# –†–∞–∑–¥–µ–ª–µ–Ω–∏–µ —Ç–µ–∫—Å—Ç–∞ –Ω–∞ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏—è
sentences = [text.split() for text in corpus]

# –°–æ–∑–¥–∞–Ω–∏–µ –º–æ–¥–µ–ª–∏ Word2Vec
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# –§—É–Ω–∫—Ü–∏—è –¥–ª—è –ø–æ–ª—É—á–µ–Ω–∏—è –≤–µ–∫—Ç–æ—Ä–∞ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è
def get_comment_vector(comment, model):
    words = comment.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# –ü—Ä–∏–º–µ—Ä –ø–æ–ª—É—á–µ–Ω–∏—è –≤–µ–∫—Ç–æ—Ä–∞ –¥–ª—è –æ–¥–Ω–æ–≥–æ –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è
comment = "–ü—Ä–∏–º–µ—Ä –∫–æ–º–º–µ–Ω—Ç–∞—Ä–∏—è –¥–ª—è –≤–µ–∫—Ç–æ—Ä–∏–∑–∞—Ü–∏–∏"
preprocessed_comment = preprocess_text(comment)
comment_vector = get_comment_vector(preprocessed_comment, word2vec_model)
print(comment_vector)
